In [1]:
!pip install pygame SpeechRecognition pyaudio


In [2]:
import pygame
import random
import speech_recognition as sr
import sys
import threading
import queue


pygame 2.6.1 (SDL 2.28.4, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Initialize Pygame
pygame.init()

# Game constants
WIDTH, HEIGHT = 400, 600
FPS = 60
GRAVITY = 0.25
FLAP_STRENGTH = -7
PIPE_SPEED = 3
PIPE_GAP = 150
PIPE_FREQUENCY = 1500  # milliseconds

# Colors
WHITE = (255, 255, 255)
BLUE = (0, 120, 255)
GREEN = (0, 200, 0)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# Initialize screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Voice-Controlled Flappy Bird")
clock = pygame.time.Clock()

# Speech recognition setup
command_queue = queue.Queue()
recognizer = sr.Recognizer()
stop_listening = None


In [4]:
class Bird:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.x = 100
        self.y = HEIGHT // 2
        self.velocity = 0
        self.size = 20
        self.color = BLUE
        self.alive = True
    
    def flap(self):
        if self.alive:
            self.velocity = FLAP_STRENGTH
    
    def update(self):
        if not self.alive:
            return
            
        self.velocity += GRAVITY
        self.y += self.velocity
        
        if self.y < 0:
            self.y = 0
            self.velocity = 0
        if self.y > HEIGHT - self.size:
            self.y = HEIGHT - self.size
            self.velocity = 0
            self.alive = False
    
    def draw(self):
        points = [
            (self.x, self.y - self.size),
            (self.x - self.size, self.y + self.size),
            (self.x + self.size, self.y + self.size)
        ]
        pygame.draw.polygon(screen, self.color, points)

class Pipe:
    def __init__(self):
        self.x = WIDTH
        self.gap_y = random.randint(150, HEIGHT - 150)
        self.width = 60
        self.passed = False
    
    def update(self):
        self.x -= PIPE_SPEED
    
    def draw(self):
        pygame.draw.rect(screen, GREEN, (self.x, 0, self.width, self.gap_y - PIPE_GAP // 2))
        pygame.draw.rect(screen, GREEN, (self.x, self.gap_y + PIPE_GAP // 2, self.width, HEIGHT))
    
    def collide(self, bird):
        if not bird.alive:
            return False
            
        bird_mask = pygame.Rect(bird.x - bird.size, bird.y - bird.size, 
                              bird.size * 2, bird.size * 2)
        top_pipe = pygame.Rect(self.x, 0, self.width, self.gap_y - PIPE_GAP // 2)
        bottom_pipe = pygame.Rect(self.x, self.gap_y + PIPE_GAP // 2, self.width, HEIGHT)
        
        return bird_mask.colliderect(top_pipe) or bird_mask.colliderect(bottom_pipe)


In [5]:
def callback(recognizer, audio):
    try:
        command = recognizer.recognize_google(audio).lower()
        print(f"Heard: {command}")
        if any(word in command for word in ["flap", "jump", "up", "go", "fly", "bird"]):
            command_queue.put("flap")
    except sr.UnknownValueError:
        pass
    except Exception as e:
        print(f"Speech recognition error: {e}")

def start_listening():
    global stop_listening
    microphone = sr.Microphone()
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
    stop_listening = recognizer.listen_in_background(microphone, callback)
    print("Started listening for voice commands...")

def stop_listening_thread():
    global stop_listening
    if stop_listening:
        stop_listening(wait_for_stop=False)
        stop_listening = None
        print("Stopped listening for voice commands")


In [6]:
def game_loop():
    bird = Bird()
    pipes = []
    score = 0
    last_pipe = pygame.time.get_ticks()
    font = pygame.font.SysFont('Arial', 30)
    
    start_listening()
    
    while bird.alive:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                stop_listening_thread()
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    bird.flap()
        
        try:
            command = command_queue.get_nowait()
            if command == "flap":
                bird.flap()
        except queue.Empty:
            pass
        
        bird.update()
        current_time = pygame.time.get_ticks()
        if current_time - last_pipe > PIPE_FREQUENCY:
            pipes.append(Pipe())
            last_pipe = current_time
        
        for pipe in pipes[:]:
            pipe.update()
            if pipe.x + pipe.width < 0:
                pipes.remove(pipe)
                continue
            if pipe.collide(bird):
                bird.alive = False
            if pipe.x + pipe.width < bird.x and not pipe.passed:
                pipe.passed = True
                score += 1
        
        screen.fill(WHITE)
        bird.draw()
        for pipe in pipes:
            pipe.draw()
        
        score_text = font.render(f"Score: {score}", True, BLACK)
        screen.blit(score_text, (10, 10))
        status_text = font.render("Voice: ON", True, GREEN)
        screen.blit(status_text, (WIDTH - 100, 10))
        pygame.display.flip()
        clock.tick(FPS)
    
    stop_listening_thread()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    return
        
        screen.fill(WHITE)
        game_over_text = font.render(f"Game Over! Score: {score}", True, RED)
        restart_text = font.render("Press R to restart", True, BLACK)
        screen.blit(game_over_text, (WIDTH//2 - game_over_text.get_width()//2, HEIGHT//2 - 50))
        screen.blit(restart_text, (WIDTH//2 - restart_text.get_width()//2, HEIGHT//2 + 20))
        pygame.display.flip()
        clock.tick(FPS)


In [ ]:
def run_game():
    try:
        print("Starting game... Say 'flap', 'jump', or 'up' to make the bird fly!")
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source, duration=1)
            print("Microphone is ready!")
        
        while True:
            game_loop()
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure PyAudio is installed and your microphone is working.")
    finally:
        stop_listening_thread()

run_game()


Starting game... Say 'flap', 'jump', or 'up' to make the bird fly!
Microphone is ready!
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voice commands
Started listening for voice commands...
Stopped listening for voi

In [ ]:
!pip install path_to_downloaded_file.whl


In [ ]:
import pyaudio


In [ ]:
import speech_recognition as sr

recognizer = sr.Recognizer()
mic = sr.Microphone()

with mic as source:
    recognizer.adjust_for_ambient_noise(source)
    print("Microphone is working!")


In [ ]:
try:
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Microphone is ready!")
except Exception as mic_error:
    print(f"Microphone Error: {mic_error}")
    print("Make sure a microphone is connected and accessible.")
    return
